In [1]:
from azureml.core import Workspace, Experiment
import os

ws = Workspace.from_config()
print(ws)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

exp = Experiment(workspace=ws, name="udacity-project")
run = exp.start_logging()

print('\nFiles and directories:')
fd = os.getcwd
os.listdir(os.curdir)

Workspace.create(name='quick-starts-ws-227324', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-227324')
Workspace name: quick-starts-ws-227324
Azure region: westeurope
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-227324

Files and directories:


['.amlignore',
 '.amlignore.amltmp',
 '.azureml',
 '.ipynb_aml_checkpoints',
 'automl.log',
 'automl_data',
 'conda_dependencies.yml',
 'config.json',
 'outputs',
 'train.py',
 'training',
 'udacity-project.ipynb',
 'udacity-project.ipynb.amltmp',
 '__pycache__']

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

##cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "compute-cpu-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                idle_seconds_before_scaledown=2400, 
                                                        min_nodes=0,    # when inactive
                                                        max_nodes=4)    # when busy
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(workspace=ws,
                                       name=cluster_name, 
                                       provisioning_configuration=compute_config)

# Wait for the cluster to complete, show the output log
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

print('\nCheck details about compute_targets:')
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2023-03-07T08:04:00.692424+00:00', 'createdBy': {'userObjectId': '90eaf103-1f10-4083-8ebf-b2e0d41900af', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2023-03-07T08:08:49.842994+00:00', 'state': 'Running', 'vmSize': 'STANDARD_DS3_V2'}

Check details about compute_targets:
compute-cpu-cluster ComputeInstance Succeeded


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import shutil

# Specify parameter sampler
##ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
            {
                '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
                '--max_iter': choice(20, 50, 100, 120, 150)
            }
        )


# Specify a Policy
##policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

train_script = "./training"
shutil.copy('train.py', train_script)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
##src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory=train_script, 
                      script='train.py', 
                      compute_target=compute_target, 
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
##hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 
                                     hyperparameter_sampling=ps, 
                                     policy=policy, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                     max_total_runs=4,       
                                     max_concurrent_runs=4)  

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

# Start the HyperDrive run
hyperdrive_run = exp.submit(config = hyperdrive_config, show_output = True)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

# Wait for the cluster to complete, show the output log
hyperdrive_run.wait_for_completion(show_output=True)

# evaluate the the run is indeed complete
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_60684486-8b09-4fb2-83ea-d249a00df5cf
Web View: https://ml.azure.com/runs/HD_60684486-8b09-4fb2-83ea-d249a00df5cf?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-227324/workspaces/quick-starts-ws-227324&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-07T08:46:26.316095][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-07T08:46:27.3694444Z][SCHEDULER][INFO]Scheduling job, id='HD_60684486-8b09-4fb2-83ea-d249a00df5cf_0' 
[2023-03-07T08:46:27.4905245Z][SCHEDULER][INFO]Scheduling job, id='HD_60684486-8b09-4fb2-83ea-d249a00df5cf_1' 
[2023-03-07T08:46:27.6353120Z][SCHEDULER][INFO]Scheduling job, id='HD_60684486-8b09-4fb2-83ea-d249a00df5cf_2' 
[2023-03-07T08:46:27.688584][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-07T08:46:27.7703336Z][SCHEDULER][INFO]Scheduling job, id='HD_60684486-8b09-4fb2-83ea-d249a00df5

KeyError: 'log_files'

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False)) # top=1

# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

print('\nBest run ID: ', best_run.id)

# Returns the metrics from all the runs that were launched by this HyperDriveRun.
best_run_metrics = best_run.get_metrics()
best_run_metrics

# check metrics details
print("\nBest run metrics :",best_run.get_metrics())
print('\nAccuracy:', best_run_metrics['Accuracy'])

# Returns a dictionary with the details for the run
print("\nBest run details :",best_run.get_details())
print(best_run.get_details()['runDefinition']['arguments'])

# Returns a list of the files that are stored in association with the run.
print("\nBest run file names :",best_run.get_file_names()) # get name of files of best_run

# save file  --> TODO
os.makedirs('./outputs',exist_ok = True)
##joblib.dump(value=best_run, filename = 'outputs/bestHDModel.txt')
##joblib.dump(value=best_run, filename='outputs/bankmarketing_model.pkl')
##hyperdrive_model = best_run.register_model(model_name='hyperdrive_model', 
##                                            model_path='outputs/model.joblib')

[{'run_id': 'HD_60684486-8b09-4fb2-83ea-d249a00df5cf_2', 'hyperparameters': '{"--C": 100.0, "--max_iter": 120}', 'best_primary_metric': 0.9160849772382398, 'status': 'Completed'}, {'run_id': 'HD_60684486-8b09-4fb2-83ea-d249a00df5cf_0', 'hyperparameters': '{"--C": 1.0, "--max_iter": 120}', 'best_primary_metric': 0.9160849772382398, 'status': 'Completed'}, {'run_id': 'HD_60684486-8b09-4fb2-83ea-d249a00df5cf_1', 'hyperparameters': '{"--C": 0.1, "--max_iter": 50}', 'best_primary_metric': 0.9153262518968134, 'status': 'Completed'}, {'run_id': 'HD_60684486-8b09-4fb2-83ea-d249a00df5cf_3', 'hyperparameters': '{"--C": 10.0, "--max_iter": 20}', 'best_primary_metric': 0.9098634294385433, 'status': 'Completed'}]

Best run ID:  HD_60684486-8b09-4fb2-83ea-d249a00df5cf_0

Best run metrics : {'Regularization Strength:': 1.0, 'Max iterations:': 120, 'Accuracy': 0.9160849772382398}

Accuracy: 0.9160849772382398

Best run details : {'runId': 'HD_60684486-8b09-4fb2-83ea-d249a00df5cf_0', 'target': 'compute

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=url)

# create a dataframe with ds data
ds_df = dataset.to_pandas_dataframe()
ds_df.head()
ds_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32950 non-null  int64  
 1   job             32950 non-null  object 
 2   marital         32950 non-null  object 
 3   education       32950 non-null  object 
 4   default         32950 non-null  object 
 5   housing         32950 non-null  object 
 6   loan            32950 non-null  object 
 7   contact         32950 non-null  object 
 8   month           32950 non-null  object 
 9   day_of_week     32950 non-null  object 
 10  duration        32950 non-null  int64  
 11  campaign        32950 non-null  int64  
 12  pdays           32950 non-null  int64  
 13  previous        32950 non-null  int64  
 14  poutcome        32950 non-null  object 
 15  emp.var.rate    32950 non-null  float64
 16  cons.price.idx  32950 non-null  float64
 17  cons.conf.idx   32950 non-null 

In [7]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core.dataset import Dataset

# Use the clean_data function to clean your data.
##x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dataset)

dataset.take(3).to_pandas_dataframe()

# split data into Train and Test Sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

# combine the training features and the label
train_df = pd.concat([x_train, y_train.to_frame(name="label")], axis=1)
validation_df = pd.concat([x_test, y_test.to_frame(name="label")], axis=1)

train_df.head()
train_df.info()

if not os.path.isdir('automl_data'):
    os.mkdir('automl_data')

# Save the train and validation data to a csv to be uploaded to the datastore
train_df.to_csv("automl_data/train_data.csv", index=False)
validation_df.to_csv("automl_data/validation_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./automl_data', target_path='bankmarketing', overwrite=True, show_progress=True)

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))
validation_data = Dataset.Tabular.from_delimited_files(path=ds.path('bankmarketing/train_data.csv'))

label = "label"
train_data.take(3).to_pandas_dataframe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26360 entries, 26529 to 2732
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            26360 non-null  int64  
 1   marital                        26360 non-null  int64  
 2   default                        26360 non-null  int64  
 3   housing                        26360 non-null  int64  
 4   loan                           26360 non-null  int64  
 5   month                          26360 non-null  int64  
 6   day_of_week                    26360 non-null  int64  
 7   duration                       26360 non-null  int64  
 8   campaign                       26360 non-null  int64  
 9   pdays                          26360 non-null  int64  
 10  previous                       26360 non-null  int64  
 11  poutcome                       26360 non-null  int64  
 12  emp.var.rate                   26360 non-nu

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,label
0,40,1,0,1,0,11,5,124,3,999,...,0,0,0,0,0,0,0,1,0,0
1,42,1,0,0,0,5,4,207,1,999,...,1,0,0,0,0,0,0,1,0,0
2,36,1,0,0,0,7,4,156,1,999,...,0,0,0,0,1,0,0,0,0,0


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
'''
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)
'''

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_df,
    label_column_name='y',
    n_cross_validations=5)


In [9]:
######## My Code ######
from azureml.widgets import RunDetails
#######################

# Submit your automl run

### YOUR CODE HERE ###

automl_run = exp.submit(automl_config, show_output=True)

RunDetails(automl_run).show()

# Wait to complete, show the output log
####automl_run.wait_for_completion()
automl_run.wait_for_completion(show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


DataException: DataException:
	Message: Expected column(s) y not found in X.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Expected column(s) y not found in X.",
        "target": "X",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "MissingColumnsInData"
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

# get name of files of best_run
best_run.get_file_names()
print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
best_run.get_metrics()

fitted_model
best_run

# Cluster clean up
compute_target.delete()